In [1]:
from datetime import datetime
import os,csv,re
from math import sqrt, cos, radians

weather_header = ['station_id','station_lat','station_lon','weather_date_time','temperature','dew_point','humidity','wind','wind_speed','wind_gust','pressure','precipation','condition']


In [5]:
# load weather stations data
weather_satation_data = []
for file in os.listdir('processed_stations'):
    if file.endswith('.csv'):
        with open('processed_stations/'+file, 'r') as f:
            reader = csv.reader(f)
            for i,line in enumerate(reader):
                if i == 0:
                    continue
                weather_satation_data.append(line)
weather_satation_data

[['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 00:51:00',
  '22',
  '10',
  '60',
  'SW',
  '6',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 01:51:00',
  '24',
  '12',
  '60',
  'SSW',
  '7',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 02:51:00',
  '23',
  '10',
  '58',
  'SW',
  '6',
  '0',
  '30',
  '0',
  'Partly Cloudy'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 03:51:00',
  '23',
  '10',
  '58',
  'SW',
  '7',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 04:51:00',
  '24',
  '8',
  '51',
  'SW',
  '5',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 05:51:00',
  '22',
  '8',
  '55',
  'SSW',
  '5',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',
  '2015-01-01 06:51:00',
  '21',
  '9',
  '60',
  'SSW',
  '6',
  '0',
  '30',
  '0',
  'Fair'],
 ['KNJHOBOK63',
  '40.74',
  '-74.03',


In [ ]:
def convert_to_24hour_and_contatenated_date(date,time):
    in_time = datetime.strptime(date + " " + time, "%Y-%m-%d %I:%M %p")
    out_time = datetime.strftime(in_time, "%Y-%m-%d %H:%M:%S")
    return out_time

# preprocess the station files data
# convert date and time to 24 hour format
# remove all non numeric characters

count = 0
for station_file in  os.listdir('station_data_original'):
    with open('station_data_original/'+station_file, 'r',) as file:
        reader = csv.reader(file, delimiter = ',')
       
        with open('processed_stations/'+station_file, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(weather_header)

        for i,line in enumerate(reader):
            # if i == 0:
            #     continue
            new_line = []
            date = line[3]
            time = line[4]
            date_and_time = convert_to_24hour_and_contatenated_date(date,time)
            new_line.append(line[0])
            new_line.append(line[1])
            new_line.append(line[2])
            new_line.append(date_and_time)  
            new_line.append(re.findall(r'\d+',line[5])[0])
            new_line.append(re.findall(r'\d+',line[6])[0])
            new_line.append(re.findall(r'\d+',line[7])[0])
            new_line.append(line[8])
            new_line.append(re.findall(r'\d+',line[9])[0])
            new_line.append(re.findall(r'\d+',line[10])[0])
            new_line.append(re.findall(r'\d+',line[11])[0])
            new_line.append(re.findall(r'\d+',line[12])[0])
            new_line.append(line[13])

            with open('processed_stations/'+station_file, 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(new_line)


In [ ]:
stations = []
#load stations data
with open("stations.csv", 'r',) as file:
        reader = csv.reader(file, delimiter = ',')
        for i,line in enumerate(reader):
            if i == 0:
                continue
            st = {}
            st['station_id'] = line[0]
            st['lat'] = float(line[1])
            st['lon'] = float(line[2])
            with open('processed_stations/'+line[0]+".csv", 'r') as f:
                reader = csv.reader(f)
                data = []
                for i,line in enumerate(reader):
                    if i == 0:
                        continue
                    data.append(line)
            st['data'] = data          
            stations.append(st)

stations

In [ ]:

# get distance between two points in km
def get_distance_of_two_points(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the earth in km
    x = (radians(lon2) - radians(lon1)) * cos(0.5 * (radians(lat2) + radians(lat1)))
    y = radians(lat2) - radians(lat1)
    d = R * sqrt(x*x + y*y)
    return d

# get closest satation
def get_closest_station(lat, lon, stations):
    min_distance = 100000000
    closest_station = None
    for station in stations:
        distance = get_distance_of_two_points(lat, lon, station['lat'], station['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_station = station
    return closest_station,distance


def get_weather_info_from_station_data(station_id,pickup_datetime):
    pickup_datetime = datetime.strptime(pickup_datetime, "%Y-%m-%d %H:%M:%S")
    min_time_diff = 100000000
    closest_weather = None
    for station in stations:
        if station['station_id'] == station_id:
            for weather in station['data']:
                weather_date_time = datetime.strptime(weather[3], "%Y-%m-%d %H:%M:%S")
                time_diff = abs((weather_date_time - pickup_datetime).total_seconds())
                if time_diff < min_time_diff:
                    min_time_diff = time_diff
                    closest_weather = weather
    return closest_weather    



def check_if_any_column_is_empty(row):
    for i in row:
        if i == '':
            return True
        if i is None:
            return True
    return False

with open("taxi_trip_raw_data/2015_Green_Taxi_Trip_Data.csv", 'r',) as file:
        reader = csv.reader(file, delimiter = ',')
        writer1 = None
        for i,line in enumerate(reader):
            if i == 0:
                line = [x.lower() for x in line]
                combine_header = line + weather_header
                with open('processed_trip_data/green_trip_data_2015_2.csv', 'w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(combine_header)
                
                writer1 = open('processed_trip_data/green_trip_data_2015_2.csv', 'a+', newline='')
                continue


            try:
                lon =float(line[5])
                lat = float(line[6])

                if lon != 0 or lat != 0 or check_if_any_column_is_empty(line) is False:
                    
                    in_time = datetime.strptime(line[1], "%m/%d/%Y %I:%M:%S %p")
                    pickup_datetime = datetime.strftime(in_time, "%Y-%m-%d %H:%M:%S")

                    in_time = datetime.strptime(line[2], "%m/%d/%Y %I:%M:%S %p")
                    dropoff_date_time = datetime.strftime(in_time, "%Y-%m-%d %H:%M:%S")

                    #change pickup and dropoff time
                    line[1] = pickup_datetime
                    line[2] = dropoff_date_time


                    closest_station,distance = get_closest_station(lat, lon, stations)
                    station_id = closest_station['station_id']
                    #weather_info = get_weather_information_from_station(station_id,pickup_datetime)
                    #weather_info = get_weather_information_from_weather_data(station_id,pickup_datetime)
                    weather_info = get_weather_info_from_station_data(station_id,pickup_datetime)
            
                    new_line = line+weather_info
                    my_str = ','.join(str(item) for item in new_line)
                    writer1.write(my_str + '\n')
                    # with open('processed_trip_data/green_trip_data_2015.csv', 'a', newline='') as file:
                    #     writer = csv.writer(file)
                    #     writer.writerow(new_line)


            except Exception as e:
                print(e)

        writer1.close()

print("done")            
                